In [ ]:
import sys
from time import time
import numpy as np
import scipy
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
from sklearn import neighbors
!pip install munkres
from munkres import Munkres
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn.manifold import Isomap, TSNE
from numba import jit

In [ ]:
# load MNIST data and normalization
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, data_home='mnist/')
y = np.asarray(list(map(int, y)))
X = np.asarray(X.astype(float))
X = scale(X)
n_digits = len(np.unique(y))

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score

import cupy as cp

def kmeans(X, n_clusters, max_iter=300, tol=1e-4, seed_method='pca', n_init=10):
    n, m = X.shape
    best_labels = None
    best_centroids = None
    best_inertia = cp.inf

    X_gpu = cp.asarray(X)

    for _ in range(n_init):
        pca = PCA(n_components=n_clusters)
        pca.fit(cp.asnumpy(X_gpu))
        C = cp.asarray(pca.components_)


        for _ in range(max_iter):
            distances = cp.sqrt(((X_gpu - C[:, cp.newaxis])**2).sum(axis=2))
            labels = distances.argmin(axis=0)

            new_C = cp.array([X_gpu[labels == i].mean(axis=0) for i in range(n_clusters)])

            if cp.sqrt(((C - new_C)**2).sum()) < tol:
                break
            C = new_C

        inertia = ((X_gpu - C[labels])**2).sum()

        if inertia < best_inertia:
            best_labels = labels
            best_centroids = C
            best_inertia = inertia

    best_labels = cp.asnumpy(best_labels)
    best_centroids = cp.asnumpy(best_centroids)

    return best_labels, best_centroids

def evaluate_clustering(labels, y):
    metrics = {
        'homogeneity': homogeneity_score(y, labels),
        'completeness': completeness_score(y, labels),
        'v_measure': v_measure_score(y, labels),
        'adjusted_rand': adjusted_rand_score(y, labels),
        'adjusted_mutual_info': adjusted_mutual_info_score(y, labels)
    }
    return metrics

# principal eigenvectors
labels, _ = kmeans(X, n_digits, seed_method='pca')
print("Principal Eigenvectors:")
print(evaluate_clustering(labels, y))


pca = PCA(n_components=30)
X_pca = pca.fit_transform(X)

best_objective = float('inf')
best_labels = None

for _ in range(10):
    labels, centroids = kmeans(X_pca, n_digits)
    objective = ((X_pca - centroids[labels])**2).sum()
    if objective < best_objective:
        best_objective = objective
        best_labels = labels

print("PCA:")
print(evaluate_clustering(best_labels, y))

Principal Eigenvectors:
{'homogeneity': 0.41979752691526667, 'completeness': 0.44191904209593696, 'v_measure': 0.430574338802637, 'adjusted_rand': 0.32075816979535976, 'adjusted_mutual_info': 0.43042739835433946}
PCA:
{'homogeneity': 0.41863121698886396, 'completeness': 0.4402897730221927, 'v_measure': 0.4291874238762253, 'adjusted_rand': 0.31980453045680396, 'adjusted_mutual_info': 0.4290401920442158}


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

def match_labels(z, y):
    K = len(np.unique(y))
    W = np.zeros((K, K))

    for i in range(K):
        for j in range(K):
            W[i, j] = 1 - len(set(np.where(z == i)[0]) & set(np.where(y == j)[0]))

    W = W - W.min(axis=1, keepdims=True)

    row_ind, col_ind = linear_sum_assignment(W)
    z_mapped = np.zeros_like(z)
    for i in range(K):
        z_mapped[z == row_ind[i]] = col_ind[i]

    conf_mat = confusion_matrix(y, z_mapped)
    acc = accuracy_score(y, z_mapped)

    return z_mapped, conf_mat, acc


In [ ]:
# count total possible matchings
K = 10
num_matchings = np.math.factorial(K)
print(f"Number of possible matchings: {num_matchings}")

Number of possible matchings: 3628800


<ipython-input-5-75deff716b8a>:3: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  num_matchings = np.math.factorial(K)


In [ ]:
# apply hungarian algorithm to principal eigenvectors
labels_pe, _ = kmeans(X, n_digits, seed_method='pca')
z_mapped_pe, conf_mat_pe, acc_pe = match_labels(labels_pe, y)
print("Results principal eigenvectors:")
print("Cluster mapping:")
for i in range(K):
    print(f"Cluster {i} ---> Class {z_mapped_pe[labels_pe == i][0]}")
print("Confusion matrix:")
print(conf_mat_pe)
print(f"Accuracy: {acc_pe:.4f}")

# apply hungarian algorithm to PCA
z_mapped_pca, conf_mat_pca, acc_pca = match_labels(best_labels, y)
print("Results for pca:")
print("Cluster mapping:")
for i in range(K):
    print(f"Cluster {i} -> Class {z_mapped_pca[best_labels == i][0]}")
print("Confusion matrix:")
print(conf_mat_pca)
print(f"Accuracy: {acc_pca:.4f}")

Results principal eigenvectors:
Cluster mapping:
Cluster 0 ---> Class 0
Cluster 1 ---> Class 9
Cluster 2 ---> Class 2
Cluster 3 ---> Class 6
Cluster 4 ---> Class 1
Cluster 5 ---> Class 3
Cluster 6 ---> Class 8
Cluster 7 ---> Class 7
Cluster 8 ---> Class 5
Cluster 9 ---> Class 4
Confusion matrix:
[[3876   35  118 1356   13  658  351    6  480   10]
 [   0 7638   13   27    8  159   16    5   10    1]
 [  40  827 2380  726  185   85  862   29 1815   41]
 [  10  577  734 4097  200  118   88   95 1117  105]
 [  55  536   73    5 3968  948  128  741   27  343]
 [  33  467  246 2131  311 2709  102   93  158   63]
 [ 282  561  423  109   27  133 5323    1   12    5]
 [  20  516   16    9 1580  134    3 4086   13  916]
 [  45 1171  205 2589  355 2096   31  182   78   73]
 [  44  331   23  124 3487  131    5 2368   14  431]]
Accuracy: 0.4941
Results for pca:
Cluster mapping:
Cluster 0 -> Class 0
Cluster 1 -> Class 9
Cluster 2 -> Class 2
Cluster 3 -> Class 6
Cluster 4 -> Class 1
Cluster 5 -> Cla

Problem 2biii: Number of possible matchings: 3628800

Computational complexity of Hungarian algorithm: O(K^3)

In [ ]:
import cupy as cp
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.manifold import TSNE
from scipy.sparse import coo_matrix, diags
from scipy.sparse.linalg import eigs
from sklearn.model_selection import ParameterGrid

import numpy as np

def spectral_clustering(X, y, K, k, sigma, n_init, n_samples):

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # random sample the data
    idx = np.random.choice(X_scaled.shape[0], size=n_samples, replace=False)
    X_subset = X_scaled[idx]
    y_subset = y[idx]

    # compute sparse distance matrix H using kNN graph
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='kd_tree', metric='euclidean').fit(X_subset)
    H = nbrs.kneighbors_graph(mode='distance')
    H = coo_matrix(H)

    if sigma is None:
        sigma = np.mean(H.data)
    E = coo_matrix((np.exp(-H.data**2 / (sigma**2)), H.nonzero()), shape=(n_samples, n_samples))
    E.setdiag(1)
    E = (E + E.T) / 2
    row_sums = E.sum(axis=1).A1
    row_sums_inv_sqrt = np.power(row_sums, -0.5)
    D_inv_sqrt = diags(row_sums_inv_sqrt)
    E_norm = D_inv_sqrt @ E @ D_inv_sqrt

    L = coo_matrix((n_samples, n_samples), dtype=np.float32)
    L.setdiag(1)
    L -= E_norm

    L_dense = L.toarray()
    L_gpu = cp.asarray(L_dense)
    eigvals, eigvecs = cp.linalg.eigh(L_gpu)
    eigvals = cp.asnumpy(eigvals)
    eigvecs = cp.asnumpy(eigvecs)

    idx = np.argsort(eigvals)[1:K+1]
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    eigvecs_norm = eigvecs / np.sqrt(np.sum(eigvecs**2, axis=1, keepdims=True) + 1e-10)

    kmeans = KMeans(init='k-means++', n_clusters=K, n_init=n_init)
    labels = kmeans.fit_predict(eigvecs_norm)

    return labels, y_subset, eigvecs_norm


K = 10
k = 500
sigma = 0.7
n_init = 10
n_samples = 10000

'''param_grid = {
    'k': [500],
    'sigma': np.arange(0.3, 1.3, 0.1),
    'n_init': [10]
}

grid = ParameterGrid(param_grid)

best_params = None
best_acc = 0

for params in grid:
    labels_spectral, y_subset = spectral_clustering(X_pca, y, K, k=params['k'], sigma=params['sigma'], n_init=params['n_init'], n_samples=n_samples)
    z_mapped_spectral, conf_mat_spectral, acc_spectral = match_labels(labels_spectral, y_subset)

    print(f"Current parameters: {params}, Current accuracy: {acc_spectral:.4f}")

    if acc_spectral > best_acc:
        best_acc = acc_spectral
        best_params = params

print(f"Best parameters: {best_params}, Best accuracy: {best_acc:.4f}")'''

labels_spectral, y_subset, eigvecs_norm = spectral_clustering(X_pca, y, K, k=k, sigma=sigma, n_init=n_init, n_samples=n_samples)

# Map the cluster labels to the true labels using the Hungarian algorithm
z_mapped_spectral, conf_mat_spectral, acc_spectral = match_labels(labels_spectral, y_subset)

print("Cluster mapping:")
for i in range(K):
    print(f"Cluster {i} -> Class {z_mapped_spectral[labels_spectral == i][0]}")
print("Confusion matrix:")
print(conf_mat_spectral)
print(f"Accuracy: {acc_spectral:.4f}")

print("Comparison")
print(f"Spectral Clustering Accuracy: {acc_spectral:.4f}")
print(f"K-means Accuracy: {acc_pca:.4f}")


Cluster mapping:
Cluster 0 -> Class 4
Cluster 1 -> Class 3
Cluster 2 -> Class 2
Cluster 3 -> Class 9
Cluster 4 -> Class 7
Cluster 5 -> Class 6
Cluster 6 -> Class 5
Cluster 7 -> Class 0
Cluster 8 -> Class 1
Cluster 9 -> Class 8
Confusion matrix:
[[914   0   5  45  17   4  40   0   7   0]
 [  0 561   2   8   1 517   3   0   7   2]
 [ 13  25 797  49   9  13  11   9  32   1]
 [  8  23 115 633   4   7   2  11 212  11]
 [  3  14  25   2 362  15  19  24   0 509]
 [ 27  13   8 243 287  10  21   1 223  33]
 [ 54  16  71   3   6   4 843   0   8   1]
 [ 12  36   9   3  49  29   0 769   8 114]
 [  9  17  12 207 104  28   4  10 557  43]
 [ 16  21   7  20 150   9   0 228   6 560]]
Accuracy: 0.6006
Comparison
Spectral Clustering Accuracy: 0.6006
K-means Accuracy: 0.4957


In [ ]:
pca = PCA(n_components=100)
X_pca = pca.fit_transform(X)

In [ ]:
from sklearn.metrics import accuracy_score
from scipy.sparse import csr_matrix

def knn_classifier(X_train, y_train, X_test, k):
    n_test = X_test.shape[0]
    y_pred = np.zeros(n_test, dtype=int)

    for i in range(n_test):
        distances = np.sqrt(((X_train - X_test[i])**2).sum(axis=1))
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = y_train[nearest_indices]
        y_pred[i] = np.argmax(np.bincount(nearest_labels))

    return y_pred

num_samples = 100

# Random Sampling + KNN
print("Random Sampling + KNN:")
S_random = np.random.choice(X_pca.shape[0], size=num_samples, replace=False)
X_train_random, y_train_random = X_pca[S_random], y[S_random]
X_test_random, y_test_random = X_pca[~np.isin(np.arange(len(X_pca)), S_random)], y[~np.isin(np.arange(len(y)), S_random)]

for k_knn in [1, 3, 5]:
    y_pred_random = knn_classifier(X_train_random, y_train_random, X_test_random, k_knn)
    acc_random = accuracy_score(y_test_random, y_pred_random)
    print(f"k = {k_knn}, Accuracy: {acc_random:.4f}")

# K-means + KNN
print("K-means + KNN:")
centroids_kmeans = []
for i in range(num_samples):
    cluster_samples = X_pca[best_labels == i]
    if len(cluster_samples) > 0:
        centroids_kmeans.append(np.mean(cluster_samples, axis=0))
    else:
        centroids_kmeans.append(np.zeros(X_pca.shape[1]))
centroids_kmeans = np.array(centroids_kmeans)

distances = pairwise_distances(X_pca, centroids_kmeans)
closest_indices = np.argmin(distances, axis=0)
S_kmeans = closest_indices[:num_samples]

X_train_kmeans, y_train_kmeans = X_pca[S_kmeans], y[S_kmeans]
X_test_kmeans, y_test_kmeans = X_pca[~np.isin(np.arange(len(X_pca)), S_kmeans)], y[~np.isin(np.arange(len(y)), S_kmeans)]

for k_knn in [1, 3, 5]:
    y_pred_kmeans = knn_classifier(X_train_kmeans, y_train_kmeans, X_test_kmeans, k_knn)
    acc_kmeans = accuracy_score(y_test_kmeans, y_pred_kmeans)
    print(f"k = {k_knn}, Accuracy: {acc_kmeans:.4f}")

# Spectral Clustering + KNN
print("Spectral Clustering + KNN:")
spectral_embedding = eigvecs_norm
centroids_spectral = np.zeros((num_samples, spectral_embedding.shape[1]))
for i in range(num_samples):
    cluster_samples = spectral_embedding[labels_spectral == i]
    if len(cluster_samples) > 0:
        centroids_spectral[i] = np.mean(cluster_samples, axis=0)
distances = pairwise_distances(spectral_embedding, centroids_spectral)
closest_indices = np.argmin(distances, axis=0)
S_spectral = closest_indices[:num_samples]

X_train_spectral, y_train_spectral = spectral_embedding[S_spectral], z_mapped_spectral[S_spectral]
X_test_spectral, y_test_spectral = spectral_embedding[~np.isin(np.arange(len(spectral_embedding)), S_spectral)], z_mapped_spectral[~np.isin(np.arange(len(z_mapped_spectral)), S_spectral)]

for k_knn in [1, 3, 5]:
    y_pred_spectral = knn_classifier(X_train_spectral, y_train_spectral, X_test_spectral, k_knn)
    acc_spectral = accuracy_score(y_test_spectral, y_pred_spectral)
    print(f"k = {k_knn}, Accuracy: {acc_spectral:.4f}")

Random Sampling + KNN:
k = 1, Accuracy: 0.7144
k = 3, Accuracy: 0.6623
k = 5, Accuracy: 0.6420
K-means + KNN:
k = 1, Accuracy: 0.3980
k = 3, Accuracy: 0.2177
k = 5, Accuracy: 0.1153
Spectral Clustering + KNN:
k = 1, Accuracy: 0.9669
k = 3, Accuracy: 0.3783
k = 5, Accuracy: 0.2370
